# Advanced Image Analysis with Azure AI Vision

## Overview
This notebook explores advanced features of Azure AI Vision that go beyond the basic image analysis covered in Lab 01. You'll learn about:
- Smart cropping and thumbnail generation
- Background removal and foreground matting
- Dense captioning (region-specific descriptions)
- Adult/racy/gory content detection
- Brand detection
- Celebrity and landmark recognition

## Prerequisites
- Completed Lab 01: Analyze Images
- Azure AI Vision resource with endpoint and key configured

## Setup and Imports

In [ ]:
# Install required packages if needed
# !pip install python-dotenv azure-ai-vision-imageanalysis==1.0.0 matplotlib pillow requests

In [ ]:
from dotenv import load_dotenv
import os
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
import requests
from io import BytesIO
import numpy as np

%matplotlib inline

print("✓ Libraries imported successfully")

In [ ]:
# Load configuration
load_dotenv('python/image-analysis/.env')

ai_endpoint = os.getenv('AI_SERVICE_ENDPOINT')
ai_key = os.getenv('AI_SERVICE_KEY')

# Create authenticated client
cv_client = ImageAnalysisClient(
    endpoint=ai_endpoint,
    credential=AzureKeyCredential(ai_key)
)

print("✓ Azure AI Vision client configured")

## 1. Smart Cropping for Thumbnails

Smart cropping uses AI to identify the most important region of an image and crops it intelligently to create thumbnails that preserve the main subject.

In [ ]:
# Load an image for smart cropping
image_file = 'python/image-analysis/images/person.jpg'

with open(image_file, "rb") as f:
    image_data = f.read()

# Display original image
img = Image.open(image_file)
print(f"Original image size: {img.size}")
plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.axis('off')
plt.title('Original Image')
plt.show()

In [ ]:
# Get smart crop suggestions
result = cv_client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.SMART_CROPS]
)

if result.smart_crops is not None:
    print(f"\nSmart Crop Suggestions: {len(result.smart_crops.list)}")
    
    # Display the top smart crop suggestion
    top_crop = result.smart_crops.list[0]
    print(f"\nTop suggestion - Aspect Ratio: {top_crop.aspect_ratio:.2f}")
    print(f"Bounding Box: x={top_crop.bounding_box.x}, y={top_crop.bounding_box.y}, "
          f"width={top_crop.bounding_box.width}, height={top_crop.bounding_box.height}")
    
    # Crop and display the smart crop
    img = Image.open(image_file)
    r = top_crop.bounding_box
    cropped = img.crop((r.x, r.y, r.x + r.width, r.y + r.height))
    
    plt.figure(figsize=(8, 8))
    plt.imshow(cropped)
    plt.axis('off')
    plt.title('Smart Cropped Thumbnail')
    plt.show()
    print(f"Cropped image size: {cropped.size}")

## 2. Dense Captions (Region-Specific Descriptions)

Dense captioning provides multiple captions for different regions of an image, allowing you to understand specific areas in detail.

In [ ]:
# Analyze image with dense captions
image_file = 'python/image-analysis/images/street.jpg'

with open(image_file, "rb") as f:
    image_data = f.read()

result = cv_client.analyze(
    image_data=image_data,
    visual_features=[VisualFeatures.DENSE_CAPTIONS]
)

# Display image
img = Image.open(image_file)
plt.figure(figsize=(12, 10))
plt.imshow(img)
plt.axis('off')
plt.title('Image with Dense Captions')
plt.show()

# Display dense captions
if result.dense_captions is not None:
    print(f"\n=== Dense Captions ({len(result.dense_captions.list)} regions) ===")
    for idx, caption in enumerate(result.dense_captions.list, 1):
        print(f"\n{idx}. \"{caption.text}\" (confidence: {caption.confidence:.2%})")
        if caption.bounding_box:
            r = caption.bounding_box
            print(f"   Region: ({r.x}, {r.y}) - {r.width}x{r.height}")

## 3. Background Removal

Azure AI Vision can remove backgrounds from images or create foreground matting, useful for e-commerce, profile pictures, and image editing applications.

In [ ]:
# Note: Background removal requires a REST API call
# It's not available through the standard SDK analyze method

def remove_background(image_path, mode='backgroundRemoval'):
    """
    Remove background from an image using Azure AI Vision.
    
    Args:
        image_path: Path to the input image
        mode: 'backgroundRemoval' or 'foregroundMatting'
    
    Returns:
        PIL Image with background removed
    """
    # Construct the API endpoint
    url = f"{ai_endpoint}/computervision/imageanalysis:segment?api-version=2023-02-01-preview&mode={mode}"
    
    # Read image data
    with open(image_path, 'rb') as f:
        image_data = f.read()
    
    # Make API request
    headers = {
        'Ocp-Apim-Subscription-Key': ai_key,
        'Content-Type': 'application/octet-stream'
    }
    
    response = requests.post(url, headers=headers, data=image_data)
    
    if response.status_code == 200:
        # Load the result image
        result_image = Image.open(BytesIO(response.content))
        return result_image
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Remove background from an image
image_file = 'python/image-analysis/images/person.jpg'

print("Processing background removal...")
print("Note: This feature may require preview API access")

try:
    result_image = remove_background(image_file)
    
    if result_image:
        # Display original and result side by side
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))
        
        original = Image.open(image_file)
        axes[0].imshow(original)
        axes[0].axis('off')
        axes[0].set_title('Original Image')
        
        axes[1].imshow(result_image)
        axes[1].axis('off')
        axes[1].set_title('Background Removed')
        
        plt.tight_layout()
        plt.show()
        print("✓ Background removal complete")
except Exception as e:
    print(f"Background removal not available: {e}")
    print("This feature may require preview API access or specific service configuration")

## 4. Content Moderation

Detect adult, racy, or gory content in images for content moderation purposes.

In [ ]:
# Analyze image for adult content
image_file = 'python/image-analysis/images/building.jpg'

with open(image_file, "rb") as f:
    image_data = f.read()

# Note: Adult content detection is typically done via REST API
# This is a demonstration of how to structure the call

print("\n=== Content Moderation Analysis ===")
print("Analyzing image for inappropriate content...")

# Using the REST API for adult content detection
url = f"{ai_endpoint}/vision/v3.2/analyze?visualFeatures=Adult"
headers = {
    'Ocp-Apim-Subscription-Key': ai_key,
    'Content-Type': 'application/octet-stream'
}

try:
    response = requests.post(url, headers=headers, data=image_data)
    
    if response.status_code == 200:
        result = response.json()
        adult = result.get('adult', {})
        
        print(f"\nAdult Content: {adult.get('isAdultContent', False)}")
        print(f"Adult Score: {adult.get('adultScore', 0):.3f}")
        print(f"\nRacy Content: {adult.get('isRacyContent', False)}")
        print(f"Racy Score: {adult.get('racyScore', 0):.3f}")
        print(f"\nGory Content: {adult.get('isGoryContent', False)}")
        print(f"Gory Score: {adult.get('goryScore', 0):.3f}")
        
        # Display the analyzed image
        img = Image.open(image_file)
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.axis('off')
        plt.title('Content Moderation Analysis')
        plt.show()
    else:
        print(f"Error: {response.status_code}")
except Exception as e:
    print(f"Content moderation analysis not available: {e}")

## 5. Image Analysis with Multiple Features

Combine multiple visual features in a single API call for comprehensive image analysis.

In [ ]:
def comprehensive_analysis(image_path):
    """
    Perform comprehensive image analysis with multiple features.
    
    Args:
        image_path: Path to the image file
    """
    with open(image_path, "rb") as f:
        image_data = f.read()
    
    # Request all available features
    result = cv_client.analyze(
        image_data=image_data,
        visual_features=[
            VisualFeatures.CAPTION,
            VisualFeatures.DENSE_CAPTIONS,
            VisualFeatures.TAGS,
            VisualFeatures.OBJECTS,
            VisualFeatures.PEOPLE,
            VisualFeatures.SMART_CROPS,
        ]
    )
    
    # Display the image
    img = Image.open(image_path)
    plt.figure(figsize=(12, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Comprehensive Image Analysis')
    plt.show()
    
    # Display all results
    print("\n" + "="*60)
    print("COMPREHENSIVE IMAGE ANALYSIS RESULTS")
    print("="*60)
    
    if result.caption:
        print(f"\n📝 Main Caption: {result.caption.text}")
        print(f"   Confidence: {result.caption.confidence:.2%}")
    
    if result.dense_captions:
        print(f"\n📍 Dense Captions: {len(result.dense_captions.list)} regions")
        for idx, caption in enumerate(result.dense_captions.list[:3], 1):
            print(f"   {idx}. {caption.text} ({caption.confidence:.2%})")
    
    if result.tags:
        print(f"\n🏷️  Tags ({len(result.tags.list)}):")
        top_tags = result.tags.list[:10]
        print("   " + ", ".join([f"{t.name} ({t.confidence:.0%})" for t in top_tags]))
    
    if result.objects:
        print(f"\n🎯 Objects: {len(result.objects.list)} detected")
        for obj in result.objects.list[:5]:
            print(f"   • {obj.tags[0].name} ({obj.tags[0].confidence:.2%})")
    
    if result.people:
        print(f"\n👥 People: {len(result.people.list)} detected")
        for person in result.people.list:
            if person.confidence > 0.5:
                print(f"   • Person ({person.confidence:.2%})")
    
    if result.smart_crops:
        print(f"\n✂️  Smart Crops: {len(result.smart_crops.list)} suggestions")
        for crop in result.smart_crops.list[:3]:
            print(f"   • Aspect ratio: {crop.aspect_ratio:.2f}")
    
    print("\n" + "="*60)

# Analyze an image comprehensively
image_file = 'python/image-analysis/images/street.jpg'
comprehensive_analysis(image_file)

## 6. Batch Image Analysis

Process multiple images efficiently in a batch.

In [ ]:
import glob

# Get all images in the directory
image_dir = 'python/image-analysis/images'
image_files = glob.glob(f"{image_dir}/*.jpg")

print(f"Processing {len(image_files)} images...\n")

# Analyze each image
results_summary = []

for image_path in image_files:
    image_name = os.path.basename(image_path)
    print(f"Analyzing: {image_name}")
    
    try:
        with open(image_path, "rb") as f:
            image_data = f.read()
        
        result = cv_client.analyze(
            image_data=image_data,
            visual_features=[VisualFeatures.CAPTION, VisualFeatures.TAGS]
        )
        
        caption = result.caption.text if result.caption else "N/A"
        top_tags = [t.name for t in result.tags.list[:3]] if result.tags else []
        
        results_summary.append({
            'file': image_name,
            'caption': caption,
            'top_tags': ', '.join(top_tags)
        })
        
    except Exception as e:
        print(f"  Error: {e}")
        results_summary.append({
            'file': image_name,
            'caption': 'Error',
            'top_tags': str(e)
        })

# Display summary
print("\n" + "="*80)
print("BATCH ANALYSIS SUMMARY")
print("="*80)
for summary in results_summary:
    print(f"\n{summary['file']}")
    print(f"  Caption: {summary['caption']}")
    print(f"  Tags: {summary['top_tags']}")
print("\n" + "="*80)

## Summary

In this advanced lab, you explored:
- ✓ Smart cropping for intelligent thumbnail generation
- ✓ Dense captioning for region-specific descriptions
- ✓ Background removal and foreground matting
- ✓ Content moderation for adult/racy/gory content
- ✓ Comprehensive multi-feature analysis
- ✓ Batch processing of multiple images

## Additional Resources

- [Azure AI Vision API Reference](https://learn.microsoft.com/azure/ai-services/computer-vision/)
- [Image Analysis concepts](https://learn.microsoft.com/azure/ai-services/computer-vision/concept-image-analysis)
- [Background removal documentation](https://learn.microsoft.com/azure/ai-services/computer-vision/concept-background-removal)
- [Smart cropping guide](https://learn.microsoft.com/azure/ai-services/computer-vision/how-to/generate-thumbnails)

## Next Steps

Try applying these advanced features to your own images and use cases:
- E-commerce: Smart cropping for product thumbnails
- Content moderation: Filtering inappropriate images
- Photo editing: Background removal for profile pictures
- Accessibility: Dense captions for detailed image descriptions